In [ ]:
%reset


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
from matplotlib import pyplot as plt
from scipy import signal
from scipy.signal import butter, lfilter, freqz,filtfilt, welch,find_peaks
from scipy.fft import fft 
from scipy.special import entr
from scipy.stats import kurtosis,skew,entropy
from scipy.stats.stats import pearsonr 
from scipy.fftpack import fft
import math
import time
import seaborn as sns
import pickle
from statistics import stdev
from statistics import variance
import random
from detecta import detect_peaks
import pywt 

# Test data loading

In [ ]:
df = pd.read_csv("acc_user2.csv")
df = df.drop(['Unnamed: 0'],axis=1)
print('\n\nHead:\n\n',df.head())
print('\n\nTail:\n\n',df.tail())

print('\n\nShape: ' , df.shape)


# Checking and clearing null and duplicate values

In [ ]:

print('\n\nNumber of null values :' ,df.isna().sum())
print('\n\nNumber of duplicate data :',df.duplicated().sum())

In [ ]:
df.dropna(inplace=True)
dfnew=df.drop_duplicates(keep='first')
print('\n\nAfter cleaning null data and duplicate data shape is :',dfnew.shape)

# Converting timezone into UTC

In [ ]:
dfnew.datetime = pd.to_datetime(dfnew.datetime)
dfnew.set_index('datetime', inplace =True)
dfnew = dfnew.tz_convert(tz='UTC')
dfnew.reset_index(inplace=True)
user = dfnew.copy()
user_id = input('Give the User_id no : ')

user['User_id'] = int(user_id)
print('\n\nAfter changing the timzone to UTC and adding the User_id column: ')
print('\nHead :\n\n',user.head())
print('Tail :\n\n',user.tail())
print('\n Shape : \n', user.shape)


In [ ]:
user=user.sort_values(by='datetime').reset_index(drop=True)
#df.set_index('datetime',inplace=True)
print('\n\nAfter sorting by the \'datetime\' column :')
print('\nHead :\n\n',user.head())
print('\nTail :\n\n',user.tail())
print('\n\n Shape:',user.shape)

# Segmenting user data where timestamp difference is larger than 2 mins

In [ ]:
user=user.sort_values(by='datetime').reset_index(drop=True)
com = dt.timedelta(seconds=120)
user['datetime']=pd.to_datetime(user['datetime'])
d = user['datetime'].diff(periods=1)
breaki = user[(user['datetime'].diff(periods=1)>=com)] 
breaki=breaki.sort_values(by='datetime')
print('\n\nData that have \'timestamp\' difference with the previous data more than the desire value :  \n\n')
print(breaki.head())
print(breaki.tail())
print('\n\nShape :',breaki.shape)

In [ ]:
d ={}
new=0
count=0
for i in tqdm(breaki.index):
    print (i)
    d[count] = pd.DataFrame()
    d[count]=user.loc[new:i-1]
    new = i
    count+=1
    if (i==breaki.index[-1]):
        d[count] = user.loc[new:]
    

In [ ]:
for i in tqdm(range(len(d))):
    print('d',i,'\n')
    print('head:',d[i].head())
    print('tail:',d[i].tail())
    print('Shape:',d[i].shape,'\n')

In [ ]:
#d3 = pd.Timestamp('2018-06-16 04:46:01.849000+00:00')
#delt = time.mktime(d3.timetuple())
#print(delt)
vlo =0
kharap=0
#print(len(d))
for i in tqdm(range(len(d))):
    if (len(d[i])<=5):
        vlo+=1
        if (d[i]["datetime"].nunique()==1):
            kharap+=1
        else:
            continue
    else : 
        continue
print('Number of DataFrame with only one data :', kharap)
print('Number of DataFrame with less or equal 5 data :', vlo)

# Resampling

In [ ]:
#unsampled1 = user1.resample('0.05S')
#inter1 = unsampled1.interpolate(method = 'spline',order=2)
#print(inter1.head())




#####x=df3['x'].to_numpy
delt = {}
f = {}
frame=pd.DataFrame()

if breaki.empty:
    print('\n\n This user has no data with datetime differance larger than that we desired\n\n ')
    lopa = pd.Timestamp(user['datetime'].iloc[-1])- pd.Timestamp(user["datetime"].iloc[0])
    lopa = lopa.total_seconds()
    lopa = (lopa*20)+1
    lopa = math.ceil(lopa)
    print('\n\n Number of total data in section {} is {}'. format(1,lopa))
    user['datetime']=pd.to_datetime(user['datetime'])
    frame['x']= signal.resample(user['x'],lopa)
    frame['y']=signal.resample(user['y'],lopa)
    frame['z']=signal.resample(user['z'],lopa)
    frame['datetime'] = pd.date_range(start=pd.Timestamp(user["datetime"].iloc[0]),periods=lopa ,freq='0.05S')
    print('\n\n Number of row of datetime of section {0} is {1} '.format(1,frame.datetime.nunique()))
    print('\nSection : 1\n')
    print('\n\n Head : \n\n',frame.head())
    print('\n\n Tail : \n\n',frame.tail())
    print('\n Shape:',frame.shape)
else :
    
    for i in tqdm(range(len(d))):
        
        if (pd.Timestamp(d[i]['datetime'].iloc[-1]) != pd.Timestamp(d[i]["datetime"].iloc[0])):
        
            delt[i] = pd.Timestamp(d[i]['datetime'].iloc[-1])- pd.Timestamp(d[i]["datetime"].iloc[0])
            delt[i] = delt[i].total_seconds()
            delt[i] = (delt[i]*20)+1
            delt[i]=math.ceil(delt[i])
            print('\n\n Number of total data in section {} is {}'. format(i,delt[i]))
            f[i] = pd.DataFrame()
            d[i]['datetime']=pd.to_datetime(d[i]['datetime'])
        
            #(f[i]['x'],f[i]['datetime'])= signal.resample(d[i]['x'],delt[i],d[i]['datetime'])
            f[i]['x']= signal.resample(d[i]['x'],delt[i])
            f[i]['y']=signal.resample(d[i]['y'],delt[i])
            f[i]['z']=signal.resample(d[i]['z'],delt[i])
            #f[i]['User_id'] = d[i]['User_id']
            f[i]['datetime'] = pd.date_range(start=pd.Timestamp(d[i]["datetime"].iloc[0]),periods=delt[i] ,freq='0.05S')
            print('\n\n Number of row of datetime of section {0} is {1} '.format(i,f[i].datetime.nunique()))
            print('\nSection', i ,'\n')
            print('\n\n Head : \n\n',f[i].head())
            print('\n\n Tail : \n\n',f[i].tail())
            print('\n Shape:',f[i].shape)
        
        elif (pd.Timestamp(d[i]['datetime'].iloc[-1]) == pd.Timestamp(d[i]["datetime"].iloc[0])):
            #f[i] = d[i]
            print('\n Here is  a single row')
            continue
            #print('\nSection', i+1 ,'\n')
            #print(f[i].head())
            #print(f[i].tail())
            #print('\n Shape:',f[i].shape)
        frame = frame.append([f[i]], ignore_index=False, sort=True)        
id_no = input('Give the User_id no : ')
frame['User_id'] = int(id_no)
frame.dropna(inplace=True)
frame=frame.drop_duplicates()
frame=frame.sort_values(by='datetime').reset_index(drop=True)
frame.datetime = pd.to_datetime(frame.datetime)
print('\nAfter concatanation :\n Head:\n \n \n ',frame.head())
print('\nTail:\n\n\n',frame.tail())
print('\nShape: ',frame.shape)

# Segmenting resampled data where timestamp difference is larger than 2 mins

In [ ]:
frame=frame.sort_values(by='datetime').reset_index(drop=True)
com3 = dt.timedelta(milliseconds=50)
frame['datetime']=pd.to_datetime(frame['datetime'])
d = frame['datetime'].diff(periods=1)
seg = frame[(frame['datetime'].diff(periods=1)>com3)] 
seg=seg.sort_values(by='datetime')
print('\n\nData that have \'timestamp\' difference with the previous data more than the desire value :  \n\n')
print(seg.head())
print(seg.tail())
print('\n\nShape :',seg.shape)

In [ ]:
wi ={}
new=0
count=1
for i in tqdm(seg.index):
    print (i)
    wi[count] = pd.DataFrame()
    wi[count]=frame.loc[new:i-1]
    new = i
    count+=1
    if (i==seg.index[-1]):
        wi[count] = frame.loc[new:]
print(len(wi))

In [ ]:
for i in tqdm(range(len(wi))):
    print('wi',i+1,'\n')
    print('head:',wi[i+1].head())
    print('tail:',wi[i+1].tail())
    print('Shape:',wi[i+1].shape,'\n')

# Windowing without overlapping

In [ ]:

pwd1 = 100
pwd2 = 50
overlap1 = 0
overlap2 = 0
final_window = pd.DataFrame(columns = ['user_id','acc_x','acc_y','acc_z','start','finish'])
final_window['acc_x'] = final_window['acc_x'].astype('object')
final_window['acc_y'] = final_window['acc_y'].astype('object')
final_window['acc_z'] = final_window['acc_z'].astype('object')
no1 = 0
no2 = pwd1
no3 = 0
no4 = pwd2
x_list = []
y_list = []
z_list = []
hal = 0
value_count = 0
data_count = 0
for j in tqdm(range(len(wi))):
    no1 = 0
    no2 = pwd1
    no3 = 0
    no4 = pwd2
    
    #val = math.ceil(((wi[j+1].index[-1]+1) - (wi[j+1].index[0]))/pwd)
    
    print('Length of wi{0} is {1}'.format(j+1, math.ceil(((wi[j+1].index[-1]+1) - (wi[j+1].index[0])))))
    wi[j+1] = wi[j+1].reset_index(drop=True)
    
    if (len(wi[j+1]) >= pwd1):
        print('\n\nThis wi{0} has more than {1} data \n\n'.format(j+1, pwd1))
        for i in tqdm(range(math.ceil(len(wi[j+1])/pwd1))):
            if no2 <= len(wi[j+1]):
                print('\n\nNumber till which list is windowed : ',no2)
                final_window.loc[hal,'start'] = wi[j+1].loc[no1,'datetime']
                final_window.loc[hal,'finish'] = wi[j+1].loc[no2-1,'datetime']
                x_list.append(wi[j+1].loc[no1:no2-1].x)
                x_list[hal] = x_list[hal].to_numpy()
                final_window.loc[hal,'acc_x'] =  x_list[hal]
                y_list.append(wi[j+1].loc[no1:no2-1].y)
                y_list[hal] = y_list[hal].to_numpy()
                final_window.loc[hal,'acc_y'] =  y_list[hal]
                z_list.append(wi[j+1].loc[no1:no2-1].z)
                z_list[hal] = z_list[hal].to_numpy()
                final_window.loc[hal,'acc_z'] =  z_list[hal]
               # final_window.loc[hal,'label'] = wi[j+1].loc[i].label
                final_window.loc[hal,'user_id'] = wi[j+1].loc[i].User_id
                no1 = no1 + pwd1
                no2 = no2 + pwd1
                print('\n\nThe \'hal\' value is : ', hal )
                hal+=1
            
            else :
                print('\n\nHere not enough data for making another window.')
                value_count += 1
                no1 = no1 + pwd1
                
                
    #elif (len(wi[j+1]) < pwd1) & (len(wi[j+1]) >= pwd2) :
     #   print('\n\nThis wi{0} has less than {1} data \n\n'.format(j+1, pwd1))
      #  while(no3 <= (len(wi[j+1]) - pwd2 + overlap2)):
      #      if no4 <= len(wi[j+1]):
      #          print('\n\nNumber till which list is windowed : ',no4)
      #          final_window.loc[hal,'start'] = wi[j+1].loc[no3,'datetime']
       #         final_window.loc[hal,'finish'] = wi[j+1].loc[no4-1,'datetime']
       #         x_list.append(wi[j+1].loc[no3:no4-1].x)
       #         x_list[hal] = x_list[hal].to_numpy()
       #         final_window.loc[hal,'acc_x'] =  x_list[hal]
       #         y_list.append(wi[j+1].loc[no3:no4-1].y)
       #         y_list[hal] = y_list[hal].to_numpy()
       #         final_window.loc[hal,'acc_y'] =  y_list[hal]
       #         z_list.append(wi[j+1].loc[no3:no4-1].z)
       #         z_list[hal] = z_list[hal].to_numpy()
       #         final_window.loc[hal,'acc_z'] =  z_list[hal]
       #         final_window.loc[hal,'label'] = wi[j+1].loc[i].label
       #         final_window.loc[hal,'user_id'] = wi[j+1].loc[i].User_id
       #         no3 = no3 + overlap2
       #         no4 = no4 + overlap2
       #         print('\n\nThe \'hal\' value is : ', hal )
       #         hal+=1
            
        #    else :
        #        print('\n\nHere not enough data for making another window.')
        #        value_count += 1
        #        no3 = no3 + overlap2
                
    else :
        data_count += 1
        
print('\n\nNo of Dataframe which has data less than desired value (50 & 100 ) : ', data_count)        
print('\n\nNo of incident when there were not sufficient data for making new widow : ', value_count )                

In [ ]:
final_window

In [ ]:
pickel_out1 = open('window22.pickle','wb')
pickle.dump(final_window,pickel_out1)
pickel_out1.close()

# Plotting user data after and before resampling

In [ ]:
time = np.linspace(0,frame.index[-1]+1,frame.index[-1]+1)
plt.rcParams['figure.figsize']=(20,10)
plt.figure()
#plt.subplot(1,3,1)
plt.plot(time,frame.iloc[0:(frame.index[-1]+1)].x.values,'r.-',markersize=5,label='X')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,2)
##plt.plot(time,dff22.iloc[20000:21000].y.values,'b.-',markersize=5,label='Y')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,3)
##plt.plot(time,dff22.iloc[20000:21000].z.values,'g.-',markersize=5,label='Z')
plt.legend(loc='best')
ax = plt.gca()
ax.set_ylim([-30,30])
ax.set_xlim([0,frame.index[-1]+1])
plt.show()

In [ ]:
time = np.linspace(0,user.index[-1]+1,user.index[-1]+1)
plt.rcParams['figure.figsize']=(20,10)
plt.figure()
#plt.subplot(1,3,1)
plt.plot(time,user.iloc[0:user.index[-1]+1].x.values,'r.-',markersize=5,label='X')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,2)
##plt.plot(time,dff22.iloc[20000:21000].y.values,'b.-',markersize=5,label='Y')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,3)
##plt.plot(time,dff22.iloc[20000:21000].z.values,'g.-',markersize=5,label='Z')
plt.legend(loc='best')
ax = plt.gca()
ax.set_ylim([-25,25])
ax.set_xlim([0,user.index[-1]+1])
plt.show()